# Import data

In [51]:
import numpy as np
import pandas as pd
import pycountry
import re

In [44]:
df = pd.read_csv("../data/bbc_science_and_climate_articles_2010-2023.csv")

# Method 1: Find location based on list of countries/states within the US 

In [195]:
# #Extracting US state information from pycountry.
# states_info = [{'state_code': subdiv.code.split('-')[1], 'state_name': subdiv.name} for subdiv in pycountry.subdivisions.get(country_code='US')]

# # Creating a DataFrame from the list of dictionaries.
# df_states = pd.DataFrame(states_info)

# # Display the first few rows of the DataFrame.
# print(df_states.head())

In [199]:
# # Creating a dictionary to hold each column as a separate list
# states_as_lists = {col_name: df_states[col_name].tolist() for col_name in df_states.columns}


In [201]:
# country_code2 = columns_as_lists['alpha_2']
# country_code3 = columns_as_lists['alpha_3']
# state_name = states_as_lists['state_name']
# state_code = states_as_lists['state_code']

## If title/description matches any item in country/state list, create column identifying location that matches

In [10]:
# pttrn = f"({'|'.join(country_name)})"
# title_country = df['title'].str.extract(pttrn, flags=re.IGNORECASE, expand=False)
# df['title_country'] = title_country[0]

# pttrn = f"({'|'.join(state_name)})"
# df['title_state'] = df['title'].str.extract(pttrn, flags=re.IGNORECASE, expand=False)

# pttrn = f"({'|'.join(country_name)})"
# descrip_country = df['description'].str.extract(pttrn, flags=re.IGNORECASE, expand=False)
# df['descrip_country'] = descrip_country[0]

# pttrn = f"({'|'.join(state_name)})"
# df['descrip_state'] = df['description'].str.extract(pttrn, flags=re.IGNORECASE, expand=False)

# Method 2: Use NLP to detect location

In [89]:
df['title'] = df['title'].astype(str)
df['description'] = df['description'].astype(str)

In [90]:
import spacy

# Load the SpaCy model
nlp = spacy.load('en_core_web_sm')

# Extract location entities
#GPE = geopolitical entities (cities/countries)
#LOC = non-GPE, mountain ranges/bodies of water

def extract_locations(text):
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    return ', '.join(locations)

# Apply the function to the 'Text' column and create a new column 'Locations' with the results
df['nlp_title'] = df['title'].apply(extract_locations)
df['nlp_descrip'] = df['description'].apply(extract_locations)

# Display the updated DataFrame
display(df)

,Unnamed: 0.1,Unnamed: 0,title,link,description,published,nlp_title,nlp_descrip
0,0,0,Mars rover's wind sensor damaged,http://www.bbc.co.uk/news/science-environment-...,A sensor on the mast of Nasa's Curiosity rover...,"Tue, 21 Aug 2012 20:03:54 GMT",Mars,
1,1,1,Bugs sunbathe to 'stay healthy',http://www.bbc.co.uk/nature/19319086,Western boxelder bugs sunbathe to fight off ge...,"Wed, 22 Aug 2012 08:08:03 GMT",,
2,2,2,Star is caught devouring planet,http://www.bbc.co.uk/news/science-environment-...,Astronomers spot evidence of a distant star co...,"Tue, 21 Aug 2012 15:55:38 GMT",,Earth
3,3,3,Arctic ice set to hit record low,http://www.bbc.co.uk/news/science-environment-...,Arctic sea ice looks set to reach a record low...,"Tue, 21 Aug 2012 10:55:46 GMT",Arctic,"Arctic sea, US"
4,4,4,Shanghai tops 'flood risk list',http://www.bbc.co.uk/news/science-environment-...,Shanghai is the most vulnerable major city in ...,"Tue, 21 Aug 2012 06:58:06 GMT",Shanghai,Shanghai
...,...,...,...,...,...,...,...,...
20097,20097,193063,Signs of spring 'shifting' in trees,http://www.bbc.co.uk/news/science-environment-...,Scientists say climate change will lead to a s...,"Mon, 02 Mar 2015 10:54:47 GMT",,
20098,20098,193066,SpaceX launches electric satellites,http://www.bbc.co.uk/news/science-environment-...,US rocket company SpaceX performs a dual-launc...,"Mon, 02 Mar 2015 13:33:28 GMT",,US
20099,20099,193088,VIDEO: Baby orangutans need new mum,http://www.bbc.co.uk/news/science-environment-...,Staff are hoping two Sumatran orangutans will ...,"Mon, 02 Mar 2015 09:47:13 GMT",,Dorset
20100,20100,193089,VIDEO: Astronauts brave week's third spacewalk,http://www.bbc.co.uk/news/science-environment-...,Two American astronauts have ventured out on a...,"Sun, 01 Mar 2015 17:18:44 GMT",,


In [91]:
#Save file as a csv file (it takes 7 min to rerun the code)
df.to_csv("../data/articles_locations_parsed.csv")

#Import file again
df = pd.read_csv("../data/articles_locations_parsed.csv")

In [92]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,link,description,published,nlp_title,nlp_descrip
0,0,0,0,Mars rover's wind sensor damaged,http://www.bbc.co.uk/news/science-environment-...,A sensor on the mast of Nasa's Curiosity rover...,"Tue, 21 Aug 2012 20:03:54 GMT",Mars,NaN
1,1,1,1,Bugs sunbathe to 'stay healthy',http://www.bbc.co.uk/nature/19319086,Western boxelder bugs sunbathe to fight off ge...,"Wed, 22 Aug 2012 08:08:03 GMT",NaN,NaN
2,2,2,2,Star is caught devouring planet,http://www.bbc.co.uk/news/science-environment-...,Astronomers spot evidence of a distant star co...,"Tue, 21 Aug 2012 15:55:38 GMT",NaN,Earth
3,3,3,3,Arctic ice set to hit record low,http://www.bbc.co.uk/news/science-environment-...,Arctic sea ice looks set to reach a record low...,"Tue, 21 Aug 2012 10:55:46 GMT",Arctic,"Arctic sea, US"
4,4,4,4,Shanghai tops 'flood risk list',http://www.bbc.co.uk/news/science-environment-...,Shanghai is the most vulnerable major city in ...,"Tue, 21 Aug 2012 06:58:06 GMT",Shanghai,Shanghai


In [155]:
#10,172 out of 20,102 have location data
df1 = df[df['nlp_title'].notna() | df['nlp_descrip'].notna()]

#4598 titles have location data
titles = df1[df1['nlp_title'].notna()]

#9,060 descriptions have location data
descriptions = df1[df1['nlp_descrip'].notna()]

In [156]:
#Import US cities dataframe
us_locations = pd.read_csv("../data/uscities.csv")

#Use only city, state_name, and county_name
city_list = us_locations['city'].unique().tolist()
state_list = us_locations['state_name'].unique().tolist()
county_list = us_locations['county_name'].unique().tolist()

us_list = ['US', 'USA', 'United States of America', 'America']

#20,721 unique cities
#52 unique states (list includes Puerto Rico & DC)
#1,906 unique counties
#237,536 unique locations

combined_list = city_list + state_list + county_list + us_list


In [203]:
#Find titles that match location list

def match_city(row):
    if row['nlp_title'] in city_list:
        return row['nlp_title']
    elif row['nlp_descrip'] in city_list:
        return row['nlp_descrip']
    else:
        return np.nan
    
def match_state(row):
    if row['nlp_title'] in state_list:
        return row['nlp_title']
    elif row['nlp_descrip'] in state_list:
        return row['nlp_descrip']
    else:
        return np.nan
    
def match_county(row):
    if row['nlp_title'] in county_list:
        return row['nlp_title']
    elif row['nlp_descrip'] in county_list:
        return row['nlp_descrip']
    else:
        return np.nan
    

#Create columns for city, state, county match
df1['city_match'] = df1.apply(match_city, axis=1)
df1['state_match'] = df1.apply(match_state, axis=1)
df1['county_match'] = df1.apply(match_county, axis=1)

#Not doing US because it's too broad

/var/folders/7l/gqvz1z110d948d7cyyfdcr7w0000gn/T/ipykernel_41796/2933158019.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['city_match'] = df1.apply(match_city, axis=1)
/var/folders/7l/gqvz1z110d948d7cyyfdcr7w0000gn/T/ipykernel_41796/2933158019.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['state_match'] = df1.apply(match_state, axis=1)
/var/folders/7l/gqvz1z110d948d7cyyfdcr7w0000gn/T/ipykernel_41796/2933158019.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [204]:
#Subset if data frame has at least one city_match, state_match, or county_match
# df2 = df1[df1['city_match'].notna() | df1['state_match'].notna() | df1['county_match'].notna()]
df3 = df1[df1['city_match'].notna()]

#2,978 have at least a city match, state match, or county match
#2,842 have a city match

In [205]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 20096
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.2  2842 non-null   int64 
 1   Unnamed: 0.1  2842 non-null   int64 
 2   Unnamed: 0    2842 non-null   int64 
 3   title         2842 non-null   object
 4   link          2842 non-null   object
 5   description   2842 non-null   object
 6   published     2842 non-null   object
 7   nlp_title     1493 non-null   object
 8   nlp_descrip   2535 non-null   object
 9   city_match    2842 non-null   object
 10  state_match   183 non-null    object
 11  county_match  514 non-null    object
dtypes: int64(3), object(9)
memory usage: 288.6+ KB


In [174]:
#Merge df3 with lat/long data set
df3.head()
df4 = df3[['title', 'link', 'description', 'published', 'city_match']]

In [175]:
df4.head()

,title,link,description,published,city_match
0,Mars rover's wind sensor damaged,http://www.bbc.co.uk/news/science-environment-...,A sensor on the mast of Nasa's Curiosity rover...,"Tue, 21 Aug 2012 20:03:54 GMT",Mars
2,Star is caught devouring planet,http://www.bbc.co.uk/news/science-environment-...,Astronomers spot evidence of a distant star co...,"Tue, 21 Aug 2012 15:55:38 GMT",Earth
7,Nasa selects another Mars mission,http://www.bbc.co.uk/news/science-environment-...,Just two weeks after landing its Curiosity rov...,"Mon, 20 Aug 2012 20:39:33 GMT",Mars
16,Nasa's Mars rover zaps first rock,http://www.bbc.co.uk/news/science-environment-...,Nasa's Curiosity rover deploys its laser instr...,"Mon, 20 Aug 2012 11:32:28 GMT",Mars
19,Scott's ship found off Greenland,http://www.bbc.co.uk/news/science-environment-...,The wreck of the ship that carried Captain Rob...,"Thu, 16 Aug 2012 14:52:21 GMT",Greenland


In [185]:
#30844 locations
len(us_locations)

#20721 unique locations
len(us_locations['city'].unique())

#Drop cities that have duplicate keys (same city name but different locality)
us_cities_filtered = us_locations[us_locations['city'].duplicated(keep=False) == False]

#16,807 cities that have unique names

#Keep important columns
us_cities_filtered = us_cities_filtered[['city', 'state_name', 'county_name', 'lat', 'lng']]


In [186]:
us_cities_filtered.head()

,city,state_name,county_name,lat,lng
0,New York,New York,Queens,40.6943,-73.9249
2,Chicago,Illinois,Cook,41.8375,-87.6866
12,Seattle,Washington,King,47.6211,-122.3244
13,San Francisco,California,San Francisco,37.7558,-122.4449
19,Queens,New York,Queens,40.7498,-73.7976


In [211]:
#Rename df4 column
df4 = df4.rename(columns = {'city_match':'city'})

#Merge df4 with us_cities_filtered based on 
df5 = df4.merge(us_cities_filtered, on='city', how='inner')


In [212]:
#Remove if city matches a country name or if it's a planet name (there are articles with Mars and Earth as a location)
# Creating a list of dictionaries, where each dictionary contains information about a country.
countries_info = []
for country in pycountry.countries:
    countries_info.append({
        'country_name': country.name,
        'alpha_2': country.alpha_2,
        'alpha_3': country.alpha_3,
        'official_name': getattr(country, 'official_name', country.name)  # Some countries might not have an official_name attribute.
    })

# Creating a DataFrame from the list of dictionaries.
df_countries = pd.DataFrame(countries_info)

countries_as_lists = {col_name: df_countries[col_name].tolist() for col_name in df_countries.columns}

country_name = countries_as_lists['country_name']

planet_name = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']


In [213]:
df6 = df5[~df5['city'].isin(country_name)]
final_df = df6[~df6['city'].isin(planet_name)]

In [216]:
len(final_df)

535

In [215]:
final_df['city'].unique()

array(['Isle', 'England', 'Rugby', 'Sardinia', 'Tripoli', 'Russia',
       'Edinburgh', 'Oklahoma', 'Stirling', 'North Wales', 'Mount Etna',
       'Gulf', 'North Sea', 'New York', 'Queensland', 'North Pole',
       'Farmland', 'Chicago', 'Bolivia', 'Hereford', 'New Orleans',
       'Northumberland', 'Perth', 'Munich', 'Amazonia', 'Sun',
       'Johannesburg', 'Belfast', 'Birdsong', 'Suffolk', 'Patagonia',
       'Seattle', 'Island', 'Santiago', 'Tennessee', 'Herculaneum',
       'Truro', 'Cape Canaveral', 'San Francisco', 'San Cristobal',
       'Brussels', 'Hampshire', 'Everest', 'Wimbledon', 'Louisiana',
       'Gulf Stream', 'Zephyr', 'Midwest', 'Brisbane', 'Donegal',
       'Dumfries', 'Maine', 'Edmonton', 'North East', 'Whitehaven',
       'Nile', 'Carmichael', 'Virgin', 'Amanda', 'Marine', 'Tulsa',
       'Baird', 'Ohio', 'Theresa', 'West End'], dtype=object)